I'm using the six subcategories of the dataframe here, we create this dataframe below so that in the second part of the note bookl this is the dataframe we are creating features for. 

In [2]:
import pandas as pd

# Go up one level, then into Data/
df = pd.read_csv('../Data/amazon_merged.csv')

# Check it's loaded
print(df.shape)



/var/folders/zz/p68469x91nz101jr8rs456_80000gq/T/ipykernel_48154/4144929710.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Data/amazon_merged.csv')


(633883, 21)


In [3]:
def extract_main_category(category_str):
    if isinstance(category_str, str):
        parts = [p.strip().strip("'\"[]") for p in category_str.split(',')]
        if len(parts) > 1:
            return parts[1]
    return None

# Apply to your dataframe
df['main_category'] = df['category'].apply(extract_main_category)

# View cleaned, unique main categories
main_cats = df['main_category'].dropna().unique()
main_cats = sorted(set(main_cats))  # Sorted and deduplicated

print(f"Found {len(main_cats)} unique main categories:")
for cat in main_cats:
    print("-", cat)


Found 21 unique main categories:
- Action Figures & Statues
- Arts & Crafts
- Baby & Toddler Toys
- Building Toys
- Dolls & Accessories
- Dress Up & Pretend Play
- Games
- Grown-Up Toys
- Hobbies
- Kids' Electronics
- Kids' Furniture
- Learning & Education
- Novelty & Gag Toys
- Party Supplies
- Puppets
- Puzzles
- Sports & Outdoor Play
- Stuffed Animals & Plush Toys
- Toy Remote Control & Play Vehicles
- Tricycles
- Video Games


In [4]:

# Count listings per main category, including NaN
main_category_counts = df['main_category'].value_counts(dropna=False)
print("\nListings per main category (including NaN):")
print(main_category_counts)




Listings per main category (including NaN):
main_category
Action Figures & Statues              77363
Games                                 66939
None                                  62243
Hobbies                               60822
Stuffed Animals & Plush Toys          57062
Dolls & Accessories                   37534
Party Supplies                        35295
Toy Remote Control & Play Vehicles    34643
Arts & Crafts                         27582
Puzzles                               27179
Dress Up & Pretend Play               24239
Sports & Outdoor Play                 22733
Novelty & Gag Toys                    21840
Learning & Education                  19254
Grown-Up Toys                         15570
Building Toys                         15192
Baby & Toddler Toys                   13357
Kids' Electronics                      8674
Tricycles                              3151
Puppets                                2209
Kids' Furniture                        1001
Video Games      

In [5]:
# Count match == 1 per main category, including NaN
match_counts = df[df['match'] == 1]['main_category'].value_counts(dropna=False)
print("\nNumber of match == 1 per main category (including NaN):")
print(match_counts)
print("\nProportion of match == 1 in the whole data (including NaN):")
print(match_counts.sum()/ len(df)*100)


Number of match == 1 per main category (including NaN):
main_category
Dolls & Accessories                   198
Arts & Crafts                         171
Toy Remote Control & Play Vehicles    149
Baby & Toddler Toys                   133
Tricycles                             126
Sports & Outdoor Play                 125
Stuffed Animals & Plush Toys           97
Learning & Education                   89
Dress Up & Pretend Play                79
None                                   74
Kids' Electronics                      58
Hobbies                                54
Action Figures & Statues               50
Novelty & Gag Toys                     32
Building Toys                          25
Puzzles                                13
Games                                  12
Grown-Up Toys                          10
Party Supplies                          9
Puppets                                 3
Name: count, dtype: int64

Proportion of match == 1 in the whole data (including NaN):
0.

In [6]:
# Calculate and print percentage of match == 1 per main category (ascending order)
percentage_match = (match_counts / main_category_counts * 100).round(2)
percentage_match_sorted = percentage_match.sort_values()

print("\nPercentage of match == 1 per main category (ascending):")
print(percentage_match_sorted)



Percentage of match == 1 per main category (ascending):
main_category
Games                                 0.02
Party Supplies                        0.03
Puzzles                               0.05
Action Figures & Statues              0.06
Grown-Up Toys                         0.06
Hobbies                               0.09
None                                  0.12
Puppets                               0.14
Novelty & Gag Toys                    0.15
Building Toys                         0.16
Stuffed Animals & Plush Toys          0.17
Dress Up & Pretend Play               0.33
Toy Remote Control & Play Vehicles    0.43
Learning & Education                  0.46
Dolls & Accessories                   0.53
Sports & Outdoor Play                 0.55
Arts & Crafts                         0.62
Kids' Electronics                     0.67
Baby & Toddler Toys                   1.00
Tricycles                             4.00
Kids' Furniture                        NaN
Video Games               

In [7]:
# Get the categories with more than 0.5% = last 6 categories with the lowest percentage of match == 1
percentage_match_no_nan = percentage_match_sorted.dropna()
highest_6_categories = percentage_match_no_nan.tail(6).index

# Sum the total number of listings in those categories
listings_in_highest_6 = main_category_counts[highest_6_categories].sum()

print(f"\nTotal number of listings in the 6 categories with the highest match == 1 percentages: {listings_in_highest_6}")



Total number of listings in the 6 categories with the highest match == 1 percentages: 113031


In [8]:
# Step 1: Filter to only rows in the 6 highest categories
df_top6 = df[df['main_category'].isin(highest_6_categories)].copy()

# Step 2: Drop rows where ASIN is not unique
asin_counts = df_top6['asin'].value_counts()
unique_asins = asin_counts[asin_counts == 1].index

df_top6_unique = df_top6[df_top6['asin'].isin(unique_asins)].copy()

# Step 3: Show results
print(f"\nInitial rows in top 6 categories: {len(df_top6)}")
print(f"Rows after keeping only unique ASINs: {len(df_top6_unique)}")
print(f"Percentage of 1's in 'match' column after filtering: {df_top6_unique['match'].mean() * 100:.2f}%")
# Save the filtered DataFrame to a new CSV file
df_top6_unique.to_csv('../Data/amazon_top6_unique.csv', index=False)
# Print the first few rows of the filtered DataFrame


Initial rows in top 6 categories: 113031
Rows after keeping only unique ASINs: 109011
Percentage of 1's in 'match' column after filtering: 0.66%
